In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pandas as pd
import numpy as np
import keras
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def NN(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized

    # define model (with dropout)
    initializer = keras.initializers.HeNormal(seed=1)

    regressor_peak = keras.models.Sequential()
    regressor_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(1, kernel_initializer=initializer))

    # Compile model

    opt = keras.optimizers.Adam(learning_rate=space['x3'])
    regressor_peak.compile(loss='mean_squared_error', optimizer= opt)

    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe UV Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)


    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y = df['Peak_eV']

    from sklearn.model_selection import GroupShuffleSplit 

    splitter = GroupShuffleSplit(test_size=.1, n_splits=2, random_state = 1)
    split = splitter.split(df, groups=df['Expt. No'])
    train_inds, test_inds = next(split)

    train = df.iloc[train_inds]
    test = df.iloc[test_inds]

    X_train = train[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y_train = train['Peak_eV']

    X_test = test[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y_test = test['Peak_eV']

    X_train = K.constant(X_train)
    y_train = K.constant(y_train)
    X_test = K.constant(X_test)
    y_train = K.constant(y_train)
            
    matrix = np.zeros([10,13])
    i = 0
    j = 0
        
    #reset weights of model
    regressor_peak.load_weights('initialize_weights3.h5')
    
    #prediction for Peak
    regressor_peak.fit(X_train, y_train, epochs=int(space['x2']), validation_data=(X_test, y_test),verbose = 0, use_multiprocessing=False, workers = 100)
    y_pred = regressor_peak.predict(X_test)

    #errors in eV
    matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
    matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
    matrix[10*i + j,2] = rae(y_test, y_pred)
    matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = 10*matrix.mean(0)[:8]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0])
    print("R2 (norm, eV):", matrix_2[0,1])
    print("RAE (norm, eV):", matrix_2[0,2])
    print("RMSE (norm, eV):", matrix_2[0,3])
    print("MAE (nm):", matrix_2[0,4])
    print("R2 (nm):", matrix_2[0,5])
    print("RAE (nm):", matrix_2[0,6])
    print("RMSE (nm):", matrix_2[0,7])

    return {'loss': matrix_2[0,3], 'status': STATUS_OK }

In [ ]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = {'x1': hp.uniform('x1', 0.1,0.5),
         'x2': hp.quniform('x2', 10,600,20),
         'x3': hp.uniform('x3', 0.0001,0.01)
        }

best = fmin(NN,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

16/16 [==============================] - 0s 667us/step                                                               

16/16 [==============================] - 0s 667us/step                         | 0/200 [00:54<?, ?trial/s, best loss=

                                                                                                                       Absorption Peak
  0%|                                                                          | 0/200 [00:54<?, ?trial/s, best loss=?]Absorption Peak
                                                                                                                       MAE (norm, eV):
  0%|                                                                          | 0/200 [00:54<?, ?trial/s, best loss=?]MAE (norm, eV):
                                                                                                                       0.18010535033593875
  0%|                                                                          | 0/

                                                                                                                       38.675921045312805
  0%|▏                                             | 1/200 [01:45<3:01:57, 54.86s/trial, best loss: 0.2227522385189207]38.675921045312805
                                                                                                                       R2 (nm):
  0%|▏                                             | 1/200 [01:45<3:01:57, 54.86s/trial, best loss: 0.2227522385189207]R2 (nm):
                                                                                                                       0.33706729739133456
  0%|▏                                             | 1/200 [01:45<3:01:57, 54.86s/trial, best loss: 0.2227522385189207]0.33706729739133456
                                                                                                                       RAE (nm):
  0%|▏                                             | 1/200 [0

                                                                                                                       R2 (norm, eV):
  2%|▋                                            | 3/200 [02:13<1:46:31, 32.44s/trial, best loss: 0.21882163023986095]R2 (norm, eV):
                                                                                                                       0.33432464822447794
  2%|▋                                            | 3/200 [02:13<1:46:31, 32.44s/trial, best loss: 0.21882163023986095]0.33432464822447794
                                                                                                                       RAE (norm, eV):
  2%|▋                                            | 3/200 [02:13<1:46:31, 32.44s/trial, best loss: 0.21882163023986095]RAE (norm, eV):
                                                                                                                       0.8223393867922617
  2%|▋                                        

  2%|▉                                            | 4/200 [02:23<1:29:48, 27.49s/trial, best loss: 0.21882163023986095]0.8247378737104133
                                                                                                                       RMSE (nm):
  2%|▉                                            | 4/200 [02:23<1:29:48, 27.49s/trial, best loss: 0.21882163023986095]RMSE (nm):
                                                                                                                       50.24266231121004
16/16 [==============================] - 0s 667us/step                                                               

16/16 [==============================] - 0s 667us/step200 [02:37<1:07:44, 20.85s/trial, best loss: 0.2188216302398609

                                                                                                                       Absorption Peak
  2%|█▏                                           | 5/200 [02:37<1:07:44, 20.85s/trial, best

                                                                                                                       RMSE (norm, eV):
  3%|█▎                                           | 6/200 [04:00<1:00:30, 18.71s/trial, best loss: 0.21882163023986095]RMSE (norm, eV):
                                                                                                                       0.22019569611889006
  3%|█▎                                           | 6/200 [04:00<1:00:30, 18.71s/trial, best loss: 0.21882163023986095]0.22019569611889006
                                                                                                                       MAE (nm):
  3%|█▎                                           | 6/200 [04:00<1:00:30, 18.71s/trial, best loss: 0.21882163023986095]MAE (nm):
                                                                                                                       39.047200293889745
  3%|█▎                                           | 6/

...

 71%|██████████████████████████████▌            | 142/200 [1:48:31<43:21, 44.85s/trial, best loss: 0.21481481413072134]39.79446434071985
                                                                                                                       R2 (nm):
 71%|██████████████████████████████▌            | 142/200 [1:48:31<43:21, 44.85s/trial, best loss: 0.21481481413072134]R2 (nm):
                                                                                                                       0.2989521398819377
 71%|██████████████████████████████▌            | 142/200 [1:48:31<43:21, 44.85s/trial, best loss: 0.21481481413072134]0.2989521398819377
                                                                                                                       RAE (nm):
 71%|██████████████████████████████▌            | 142/200 [1:48:31<43:21, 44.85s/trial, best loss: 0.21481481413072134]RAE (nm):
                                                                         

                                                                                                                       R2 (norm, eV):
 72%|██████████████████████████████▉            | 144/200 [1:49:45<26:23, 28.28s/trial, best loss: 0.21481481413072134]R2 (norm, eV):
                                                                                                                       0.33701564828490116
 72%|██████████████████████████████▉            | 144/200 [1:49:45<26:23, 28.28s/trial, best loss: 0.21481481413072134]0.33701564828490116
                                                                                                                       RAE (norm, eV):
 72%|██████████████████████████████▉            | 144/200 [1:49:45<26:23, 28.28s/trial, best loss: 0.21481481413072134]RAE (norm, eV):
                                                                                                                       0.8163570779546101
 72%|██████████████████████████████▉          

 72%|███████████████████████████████▏           | 145/200 [1:51:01<34:14, 37.35s/trial, best loss: 0.21481481413072134]0.8317320467122978
                                                                                                                       RMSE (nm):
 72%|███████████████████████████████▏           | 145/200 [1:51:01<34:14, 37.35s/trial, best loss: 0.21481481413072134]RMSE (nm):
                                                                                                                       50.800086389157855
16/16 [==============================] - 0s 667us/step                                                               

16/16 [==============================] - 0s 667us/step200 [1:52:19<44:14, 49.16s/trial, best loss: 0.2148148141307213

                                                                                                                       Absorption Peak
 73%|███████████████████████████████▍           | 146/200 [1:52:19<44:14, 49.16s/trial, bes

                                                                                                                       0.8196636271651879
 74%|███████████████████████████████▌           | 147/200 [1:53:28<50:56, 57.67s/trial, best loss: 0.21481481413072134]0.8196636271651879
                                                                                                                       RMSE (norm, eV):
 74%|███████████████████████████████▌           | 147/200 [1:53:28<50:56, 57.67s/trial, best loss: 0.21481481413072134]RMSE (norm, eV):
                                                                                                                       0.22450049880671846
 74%|███████████████████████████████▌           | 147/200 [1:53:28<50:56, 57.67s/trial, best loss: 0.21481481413072134]0.22450049880671846
                                                                                                                       MAE (nm):
 74%|███████████████████████████████▌        

16/16 [==============================] - 0s 1ms/step                                                                 

16/16 [==============================] - 0s 1ms/step9/200 [1:56:09<56:13, 66.15s/trial, best loss: 0.2148148141307213

                                                                                                                       Absorption Peak
 74%|████████████████████████████████           | 149/200 [1:56:09<56:13, 66.15s/trial, best loss: 0.21481481413072134]Absorption Peak
                                                                                                                       MAE (norm, eV):
 74%|████████████████████████████████           | 149/200 [1:56:09<56:13, 66.15s/trial, best loss: 0.21481481413072134]MAE (norm, eV):
                                                                                                                       0.17745859554574728
 74%|████████████████████████████████           | 149/200 [1:56:09<56:13, 66.15s/tr